In [1]:
import pandas as pd
import numpy as np

In [3]:
a = np.linspace(1,10,8)
a

array([ 1.        ,  2.28571429,  3.57142857,  4.85714286,  6.14285714,
        7.42857143,  8.71428571, 10.        ])